<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 05**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**



*   Entender e praticar com o algoritmo de análise sintática [ LALR(1) ] da ferramenta rply
*   Entender e praticar com processo de detecção de erros de análise sintática



In [ ]:
!pip install rply

**ALGORITMO DE ANÁLISE SINTÁTICA LALR(1)**

Anteriormente, implementamos um analisador sintático completo para o comando de atribuição com expressões ariméticas envolvendo números inteiros sem sinal:

\<atrib\>::= ID "=" \<expression\>

\<expression\> ::= NUMBER

       | \<expression\> "+" \<expression\>
 
       | \<expression\> "-" \<expression\>
 
       | \<expression\> "*" \<expression\>
 
       | \<expression\> "/" \<expression\>
 
       | "(" <expression> ")"

O primeiro passo foi implementar um analisador léxico para esta gramática, mostrado abaixo:


In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('EQUALS', r'=')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

O segundo passo foi implementar as classes em Python para representar os nós da árvore sintática gerada pelo analisador sintático:

In [ ]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()

class Attrib(BaseBox):
    def __init__(self, id, expression):
        self.id = id
        self.expression = expression

Finalmente, foi implementado o analisado sintático para o comando de atribuição:

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV','ID','EQUALS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

# regra <atrib>::= ID "=" <expression>

@pg.production('atrib : ID EQUALS expression')
def attrib(p):
  return Attrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Realizamos um teste com um comando de atribuição:

In [ ]:
arvore=parser.parse(lexer.lex('x=1+2*3'))
print(arvore)
print(arvore.id)
print(arvore.expression.eval())

O algoritmo de análise sintática utilizado pela ferramenta rply é o LALR(1). Esta sigla significa que:

*   o analisador irá olhar (**LOOK AHEAD**), no máximo, um token da entrada para decidir qual regra gramatical irá aplicar
*   o analisador irá analisar a entrada da esquerda para a direita **(L)** e fará as reduções o mais à direita **(R)** possível. 

Para entender como esta algoritmo funciona, vamos utilizar a ferramenta online abaixo:

https://fiona.dmcs.pl/tk/jsmachines.sourceforge.net/machines/lalr1.html



